# Columbia University - IEOR 4732: Computational Methods in Finance
# Homework 1 by Alexandre Duhamel - afd2153
<br><br>

## Problem 1 : Pricing European Call Options Using Transform Techniques

The characteristic function of the log of stock price in the **Black-Scholes framework** is given by:

$$
\mathbb{E}[e^{iu \ln S_T}] = \mathbb{E}[e^{iu s_T}]
$$

$$
= \exp \left( i (\ln S_0 + (r - q - \frac{\sigma^2}{2}) T) u - \frac{1}{2} \sigma^2 u^2 T \right)
$$

For the following parameters:

- **Spot price**: $ S_0 = 1900 $
- **Maturity**: $ T = 0.25 $ years
- **Volatility**: $ \sigma = 0.36 $
- **Risk-free interest rate**: $ r = 2.00\% $
- **Continuous dividend rate**: $ q = 1.87\% $
- **Strike range**: $ K = 2000,\: 2100,\: 2200 $
<br><br>

### Question a : **Fast Fourier Transform (FFT) - Multiple strikes and intercept**

Consider :
- $η = ∆ν = 0.25$
- $α = 0.4,\: 1.0,\: 1.4,\: 3.0$
- $N = 2n$ for $n = 9,\: 11,\: 13,\: 15$
- $ β = ln K − \frac{λN}{2}$

In [38]:
import numpy as np
import pandas as pd
import scipy.fftpack as fft

# Given parameters
S0 = 1900  # Spot price
T = 0.25   # Maturity (in years)
sigma = 0.36  # Volatility
r = 0.02  # Risk-free rate
q = 0.0187  # Continuous dividend yield
K_values = [2000, 2100, 2200]  # Strike prices

# FFT Parameters
eta = 0.25  # Step size in frequency domain
alpha_values = [0.4, 1.0, 1.4, 3.0]  # Damping factors
N_values = [2**6, 2**7, 2**8, 2**8]  # Grid sizes (N = 2^n)

In [ ]:
import numpy as np
import scipy.fftpack as fft

# Characteristic function of the log stock price in the Black-Scholes model
def characteristic_function(u):
    i = 1j  # Imaginary unit
    return np.exp(i * (np.log(S0) + (r - q - 0.5 * sigma**2) * T) * u - 0.5 * sigma**2 * u**2 * T)

# Compute ν(j)
def compute_nu(index, eta):
    return (index-1)*eta

# Compute ν(j)
def compute_lambda(N, eta):
    return (2*np.pi) / (N*eta)

# Compute beta
def compute_beta(K, lambda_, N):
    return np.log(K) - (lambda_ * N) / 2

def compute_x(N, eta, alpha, beta):
    X = np.zeros(N, dtype=complex)
    
    # Compute each element of the x vector
    for i in range(1, N + 1): 
        # Adjust eta for the first term
        if i == 1:
            etaa = eta / 2
        else:
            etaa = eta
        
        # Compute ν(i)
        v_i = compute_nu(i, eta)
        
        # Compute the characteristic function value
        phi = characteristic_function(v_i - (alpha + 1) * 1j)
        
        # Compute the x(i) term
        X[i - 1] = (etaa * np.exp(-r * T) / 
                    ((alpha + 1j * v_i) * (alpha + 1 + 1j * v_i)) *
                    np.exp(-1j * beta * v_i) * phi)
    return X

def fft_func(x, manual = False):
    if manual :
        N = len(x)
        y = np.zeros(N, dtype=complex)

        # Compute each element of y
        for m in range(1, N + 1):
            sum_result = 0   
            for j_ in range(1, N + 1): 
                exponent = -1j * 2 * np.pi * (j_ - 1) * (m - 1) / N
                sum_result += x[j_ - 1] * np.exp(exponent)
            y[m - 1] = sum_result 
    else :
        y = np.fft.fft(x)
    return y

def compute_option_price_fft(y, N, alpha, K):
    # Compute lambda
    lambda_ = compute_lambda(N, eta)
    
    # Compute beta for the grid
    beta = compute_beta(K, lambda_, N)

    # Generate log-strike values (k_m)
    k_m = beta + np.arange(N) * lambda_

    # Compute option prices for the grid of k_m
    prices = np.zeros(N)
    for m in range(1, N+1) :
        prices[m-1] = (
            np.exp(-alpha * k_m[m-1]) / np.pi *
            np.real(y[m-1])
        )

    # Interpolate to get the price corresponding to K
    option_price = np.interp(np.log(K), k_m, prices)
    return option_price

In [ ]:
results = []

# Compute option prices for all parameter combinations
for N in N_values:
    for alpha in alpha_values:
        # Step 3: Compute option prices for each strike price K
        for K in K_values:
            # Compute lambda
            lambda_ = compute_lambda(N, eta)
            
            # Compute beta using log of the first strike (ln(K))
            beta = compute_beta(K, lambda_, N)

            # Step 1: Compute x
            x = compute_x(N, eta, alpha, beta)  # Compute x

            # Step 2: Apply FFT to compute y
            y = fft_func(x)  # Compute FFT of x
            
            option_price = compute_option_price_fft(y, N, alpha, K)
            results.append([N, alpha, K, option_price])

In [ ]:
df_fft = pd.DataFrame(results, columns=["N", "Alpha", "Strike", "Option Price"])

print(' ')
print('===================')
print('Model is FFT multiple strikes')
print('-------------------')
print(df_fft.round({"Option Price": 2}))

 
Model is FFT multiple strikes
-------------------
      N  Alpha  Strike  Option Price
0    64    0.4    2000         95.38
1    64    0.4    2100         64.94
2    64    0.4    2200         43.01
3    64    1.0    2000         95.30
4    64    1.0    2100         64.88
5    64    1.0    2200         42.95
6    64    1.4    2000         95.30
7    64    1.4    2100         64.88
8    64    1.4    2200         42.96
9    64    3.0    2000         95.25
10   64    3.0    2100         64.88
11   64    3.0    2200         42.99
12  128    0.4    2000         95.33
13  128    0.4    2100         64.92
14  128    0.4    2200         43.03
15  128    1.0    2000         95.25
16  128    1.0    2100         64.83
17  128    1.0    2200         42.95
18  128    1.4    2000         95.25
19  128    1.4    2100         64.83
20  128    1.4    2200         42.95
21  128    3.0    2000         95.25
22  128    3.0    2100         64.83
23  128    3.0    2200         42.95
24  256    0.4    2000 

### Question a : **Fast Fourier Transform (FFT) - Single K direct method**



Consider :
- $η = ∆ν = 0.25$
- $α = 0.4,\: 1.0,\: 1.4,\: 3.0$
- $N = 2n$ for $n = 9,\: 11,\: 13,\: 15$
- $ β = ln K − \frac{λN}{2}$

In [123]:
# Compute lambda (log-strike step size)
def compute_lambda(eta, n):
    return 2 * np.pi / (eta * (2 ** n))

# Characteristic function of the log stock price in Black-Scholes model
def characteristic_function(u):
    i = 1j  # Imaginary unit
    return np.exp(i * (np.log(S0) + (r - q - 0.5 * sigma**2) * T) * u - 0.5 * sigma**2 * u**2 * T)

# Compute Ψ_T(ν)
def compute_psi(nu, alpha):
    i = 1j  # Imaginary unit
    phi = characteristic_function(nu - (alpha + 1) * i)
    psi = np.exp(-r * T) * phi / ((alpha + i * nu) * (alpha + 1 + i * nu))
    return psi

# Compute ν(j)
def compute_nu(index, eta):
    return (index-1)*eta

# Compute option price using FFT for a single strike
def compute_option_price_fft(N, alpha, eta, lambda_, K):
    i = 1j  # Imaginary unit
    exp_factor = np.exp(-alpha*K)/np.pi
    sum = np.exp(-i*compute_nu(1,eta)*K)*compute_psi(compute_nu(1,eta), alpha)*(eta/2)
    for j_ in range(2,N+1):
        nu = compute_nu(j_,eta)
        sum += np.exp(-i*nu*K)*compute_psi(nu, alpha)*eta
    option_price = exp_factor*np.real(sum)
    return option_price

In [124]:
# Given parameters
S0 = 1900  # Spot price
T = 0.25   # Maturity (in years)
sigma = 0.36  # Volatility
r = 0.02  # Risk-free rate
q = 0.0187  # Continuous dividend yield
K_values = [2000, 2100, 2200]  # Strike prices
eta = 0.25  # Step size in frequency domain
alpha_values = [0.4, 1.0, 1.4, 3.0]  # Damping factors
n_values = [9, 11, 13, 15]  # Power of 2 for grid sizes

In [ ]:
# Main computation loop
results = []
for n in n_values:
    N = 2 ** n 
    lambda_ = compute_lambda(eta, n)
    for alpha in alpha_values:
        for K in K_values:
            k = np.log(K)
            option_price = compute_option_price_fft(N, alpha, eta, lambda_, k)
            results.append([n, N, alpha, K, option_price])

In [ ]:
df_results = pd.DataFrame(results, columns=["n", "N", "Alpha", "Strike", "Option Price"])
df_results = df_results.sort_values(by=["n", "Alpha", "Strike"])

print(' ')
print('===================')
print('Model is FFT single strike')
print('-------------------')
print(df_results.round({"Option Price": 2}))

 
Model is FFT single strike
-------------------
     n      N  Alpha  Strike  Option Price
0    9    512    0.4    2000         95.33
1    9    512    0.4    2100         64.92
2    9    512    0.4    2200         43.03
3    9    512    1.0    2000         95.25
4    9    512    1.0    2100         64.83
5    9    512    1.0    2200         42.95
6    9    512    1.4    2000         95.25
7    9    512    1.4    2100         64.83
8    9    512    1.4    2200         42.95
9    9    512    3.0    2000         95.25
10   9    512    3.0    2100         64.83
11   9    512    3.0    2200         42.95
12  11   2048    0.4    2000         95.33
13  11   2048    0.4    2100         64.92
14  11   2048    0.4    2200         43.03
15  11   2048    1.0    2000         95.25
16  11   2048    1.0    2100         64.83
17  11   2048    1.0    2200         42.95
18  11   2048    1.4    2000         95.25
19  11   2048    1.4    2100         64.83
20  11   2048    1.4    2200         42.95
21  1

<br><br>
### Question b : **Fractional Fast Fourier Transform (FrFFT)**

Consider :
- $η = ∆ν = 0.25$
- $λ = ∆k = 0.1$
- $α = 0.4,\: 1.0,\: 1.4,\: 3.0$
- $N = 2n \:$ for $\:n = 6,\: 7,\: 8,\: 9$
- $β = ln K − \frac{λN}{2}$

In [ ]:
def compute_gamma(eta, lambda_) :
    return (eta*lambda_) / (2*np.pi)

def compute_x_frfft(N, eta, alpha, beta):
    return compute_x(N, eta, alpha, beta)

def compute_y_frfft(x, gamma):
    N = len(x)
    y = np.zeros(2 * N, dtype=complex)
    
    j = np.arange(N)
    y[:N] = x * np.exp(-1j * np.pi * gamma * j**2)
    
    # The remaining elements of y (from N to 2N-1) are already zeros
    return y

def compute_z_frfft(N, gamma):
    j = np.arange(N)  
    z = np.zeros(2 * N, dtype=complex) 

    # Compute the first N elements of z
    z[:N] = np.exp(1j * np.pi * gamma * j**2)

    # Compute the second N elements of z (reversed indices)
    j_reversed = np.arange(N - 1, -1, -1)  
    z[N:] = np.exp(1j * np.pi * gamma * j_reversed**2)

    return z

def compute_xi(y, z, fft_func):
    return ifft_func(fft_func(y) * fft_func(z))

def ifft_func(X, manual = False):
    if manual :
        N = len(X)
        x = np.zeros(N, dtype=complex)

        for j in range(N):
            sum_result = 0 
            for m in range(N): 
                exponent = 1j * 2 * np.pi * j * m / N
                sum_result += X[m] * np.exp(exponent)
            
            x[j] = sum_result / N 
    else :
        x = np.fft.ifft(X)
    return x

def compute_option_price_frfft(y, N, alpha, lambda_, beta, gamma, K):
    # Generate log-strike values (k_m)
    k_m = beta + np.arange(N) * lambda_
    
    # Compute gamma 
    gamma = compute_gamma(eta, lambda_)

    # Compute option prices for the grid of k_m
    prices = np.zeros(N)
    for m in range(1,N+1):
        exponential_term = np.exp(-1j * np.pi * gamma * (m-1)**2)
        prices[m-1] = (
            np.exp(-alpha * k_m[m-1]) / np.pi *
            np.real(exponential_term * y[m-1])
        )

    # Interpolate to get the price corresponding to K
    option_price = np.interp(np.log(K), k_m, prices)
    return option_price

In [128]:
# Given parameters
S0 = 1900  # Spot price
T = 0.25   # Maturity (in years)
sigma = 0.36  # Volatility
r = 0.02  # Risk-free rate
q = 0.0187  # Continuous dividend yield
K_values = [2000, 2100, 2200]  # Strike prices

# FrFFT Parameters
eta = 0.25  # Step size in frequency domain
lambda_ = 0.1  # Step size in log-strike domain
alpha_values = [0.4, 1.0, 1.4, 3.0]  # Damping factors
N_values = [2**6, 2**7, 2**8, 2**9]  # Grid sizes

In [ ]:
results_frfft = []

# Loop over all parameter combinations
for N in N_values:
    for alpha in alpha_values:
        for K in K_values:
            # Compute lambda
            lambda_ = 0.1 # compute_lambda(N, eta)
            
            # Compute beta using log of the first strike (ln(K))
            beta = compute_beta(K, lambda_, N)
            
            # Compute gamma
            gamma = compute_gamma(eta, lambda_)

            # Step 1: Compute x
            x = compute_x_frfft(N, eta, alpha, beta)

            # Step 2: Compute y
            y = compute_y_frfft(x, gamma)

            # Step 3: Compute z
            z = compute_z_frfft(N, gamma)

            # Step 4: Compute xi
            xi = compute_xi(y, z, fft_func)

            # Step 5: Compute option prices for each K
            option_price = compute_option_price_frfft(xi, N, alpha, lambda_, beta, gamma, K)
            results_frfft.append([N, alpha, K, option_price])

In [ ]:
df_frfft = pd.DataFrame(results_frfft, columns=["N", "Alpha", "Strike", "Option Price"])

print(' ')
print('===================')
print('Model is FrFFT (ie multiple strikes)')
print('-------------------')
print(df_frfft.round({"Option Price": 2}))

 
Model is FrFFT (ie multiple strikes)
-------------------
      N  Alpha  Strike  Option Price
0    64    0.4    2000         95.61
1    64    0.4    2100         65.61
2    64    0.4    2200         43.88
3    64    1.0    2000         95.22
4    64    1.0    2100         65.28
5    64    1.0    2200         43.65
6    64    1.4    2000         95.01
7    64    1.4    2100         65.10
8    64    1.4    2200         43.52
9    64    3.0    2000         94.38
10   64    3.0    2100         64.41
11   64    3.0    2200         42.93
12  128    0.4    2000         95.33
13  128    0.4    2100         64.92
14  128    0.4    2200         43.03
15  128    1.0    2000         95.25
16  128    1.0    2100         64.84
17  128    1.0    2200         42.95
18  128    1.4    2000         95.24
19  128    1.4    2100         64.84
20  128    1.4    2200         42.95
21  128    3.0    2000         95.24
22  128    3.0    2100         64.83
23  128    3.0    2200         42.95
24  256    0.4  

<br><br>
### Question c : **Fourier-Cosine (COS) method**

Consider values $[−1, 1]$, $[−4, 4]$, $[−8, 8]$, $[−12, 12]$ for the interval
$[a, b]$ and find thesensitivity of your results to the choice of $[a, b]$


In [77]:
# Given parameters
S0 = 1900  # Spot price
T = 0.25   # Maturity (in years)
sigma = 0.36  # Volatility
r = 0.02  # Risk-free rate
q = 0.0187  # Continuous dividend yield
K_values = [2000, 2100, 2200]  # Strike prices
intervals = [(-1, 1), (-4, 4), (-8, 8), (-12, 12)]  # COS intervals

In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import quad

def characteristic_function(u, S0, K, r, q, sigma, T):
    i = 1j
    return np.exp(i * (np.log(S0/K) + (r - q - 0.5 * sigma**2) * T) * u - 0.5 * sigma**2 * T * u**2)

def chi_k(k, c, d, a, b):
    def integrand(y):
        return np.exp(y) * np.cos(k * np.pi * (y - a) / (b - a))

    result, _ = quad(integrand, c, d)
    return result

# Define φ_k(c, d)
def phi_k(k, c, d, a, b):
    def integrand(y):
        return np.cos(k * np.pi * (y - a) / (b - a))

    # Perform the integration
    result, _ = quad(integrand, c, d)
    return result

# Compute V_k for call and put options
def V_k(k, a, b, K, option_type="int"):
    if option_type == "int":
        # For a call payoff, f(x)=K*(e^x-1) for x>=0 and zero for x < 0.
        lower = max(a, 0)  # if a < 0, start at 0 since payoff is zero below 0
        if lower >= b:
            return 0.0

        # Define the integrand for the call payoff
        def integrand(x):
            return K * (np.exp(x) - 1) * np.cos(k * np.pi * (x - a) / (b - a))
        
        result, _ = quad(integrand, lower, b)
        return 2.0 / (b - a) * result
    elif option_type == "call":
        return 2 / (b - a) * K * (chi_k(k, 0, b, a, b) - phi_k(k, 0, b, a, b))
    elif option_type == "put":
        return 2 / (b - a) * K * (-chi_k(k, a, 0, a, b) + phi_k(k, a, 0, a, b))
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

def price_option_cos(S0, K, r, q, sigma, T, N, a, b, option_type="int"):
    # Step 1: Compute the coefficients using the correct characteristic function
    u = np.arange(0, N) * np.pi / (b - a)
    phi = characteristic_function(u, S0, K, r, q, sigma, T)
    Ak = 2 / (b - a) * np.real(phi * np.exp(-1j * u * a))
    Ak[0] *= 1/2
    
    # Step 2: Compute the V_k values for the call payoff f(x) = K*(e^x-1)^+.
    # (Assuming your functions chi_k and phi_k are as defined.)
    Vk = np.array([V_k(k, a, b, K, option_type) for k in range(N)])
    
    # Step 3: Compute the option price
    option_price = ((b - a) / 2) * np.exp(-r * T) * np.sum(Ak * Vk)
    return option_price

In [ ]:
cos_results = []
for N in [50, 100, 200]:
    for (a, b) in intervals:
        for K in K_values:
            price = price_option_cos(S0, K, r, q, sigma, T, N, a, b, option_type="call")
            cos_results.append([N, a, b, K, price])

In [ ]:
df_cos_results = pd.DataFrame(cos_results, columns=["N", "a", "b", "Strike", "Option Price"])

print(' ')
print('===================')
print('Model is Cosine Series Expansion')
print('-------------------')
print(df_cos_results.round({"Option Price": 2}))

 
Model is Cosine Series Expansion
-------------------
      N   a   b  Strike  Option Price
0    50  -1   1    2000         95.25
1    50  -1   1    2100         64.83
2    50  -1   1    2200         42.95
3    50  -4   4    2000         95.21
4    50  -4   4    2100         64.72
5    50  -4   4    2200         42.85
6    50  -8   8    2000        311.21
7    50  -8   8    2100       -289.10
8    50  -8   8    2200       -828.43
9    50 -12  12    2000      83755.93
10   50 -12  12    2100      15739.00
11   50 -12  12    2200     -56808.36
12  100  -1   1    2000         95.25
13  100  -1   1    2100         64.83
14  100  -1   1    2200         42.95
15  100  -4   4    2000         95.25
16  100  -4   4    2100         64.83
17  100  -4   4    2200         42.95
18  100  -8   8    2000         96.48
19  100  -8   8    2100         67.71
20  100  -8   8    2200         45.18
21  100 -12  12    2000        466.95
22  100 -12  12    2100       4798.42
23  100 -12  12    2200       753

### Sensitivity of COS Method to the Truncation Interval [a, b]

- **Stable Results for Narrow Intervals:**  
  When using narrow intervals such as **[-1, 1]** or **[-4, 4]**, the COS method produces stable and consistent option prices. For example, the computed prices (≈95.25 for K=2000, ≈64.83 for K=2100, ≈42.95 for K=2200) align well with FFT/FrFFT results. This indicates that when the truncation interval adequately captures the bulk of the probability density of $ x = \ln(S_T/K) $, the cosine expansion accurately represents the payoff function.

- **Instability with Wider Intervals:**  
  Widening the interval to **[-8, 8]** or **[-12, 12]** leads to numerical instability:
  - **Erratic Pricing:**  
    Prices can become extremely high or even negative, deviating significantly from theoretical expectations.
  - **Underlying Cause:**  
    The exponential term $ e^x $ in the payoff grows rapidly in the tails, causing the payoff coefficients $ V_k $ in the COS expansion to be mis-scaled. This mis-scaling results in significant numerical errors when the tails are not properly truncated.

- **Additional Considerations:**  
  - **Flexibility for Different Payoffs:**  
    One notable advantage of the COS method is its versatility—it can be easily adapted to compute prices for both call and put options by simply switching the corresponding payoff function. This makes it particularly useful when a unified framework is needed for pricing different types of options.
  - **Calibration and Care:**  
    Given its sensitivity to the truncation interval, extra care and calibration are necessary. Choosing an interval that closely captures the central mass of the distribution of $ x $ is essential for achieving stable and accurate option prices.

- **Conclusion:**  
  The COS method is a powerful and flexible tool for option pricing, especially when one needs to switch between call and put options. However, its accuracy heavily depends on the careful selection of the truncation interval [a, b]. Using a narrow interval (e.g., **[-1,1]** or **[-4,4]**) that captures the bulk of the density of $ x = \ln(S_T/K) $ is crucial to avoid numerical errors and to ensure reliable pricing results.

### Comparaison and conclusions :

### 1. FFT Methods (Multiple Strikes & Single Strike)
- **Consistency:**  
  Both the multiple-strike and single-strike FFT implementations yield nearly identical option prices. For example:
  - **K = 2000:** ~95.3  
  - **K = 2100:** ~64.9  
  - **K = 2200:** ~43.0

- **Grid Convergence:**  
  Increasing the grid size (N) from 64 up to 512 (or higher) has minimal impact on the final prices, demonstrating robust convergence and numerical stability.

- **Theoretical Alignment:**  
  The grid construction (ν, λ, β), the damping factors (α), and the weighting in the x vector are implemented consistently with the theoretical definitions. The interpolation step further confirms that the FFT method is both theoretically and numerically sound.

---

### 2. FrFFT
- **Comparable Results:**  
  The FrFFT method produces option prices that are nearly identical to those from the FFT methods (e.g., 95.61 vs. 95.33 for K = 2000), with only minor differences in the decimal places.

- **Robustness and Convergence:**  
  FrFFT converges well across different grid sizes and damping factors, confirming its reliability and high numerical accuracy.

---

### 3. COS Method
- **Sensitivity to [a, b]:**  
  - **Narrow Intervals:**  
    When using narrow intervals such as **[-1, 1]** or **[-4, 4]**, the COS method produces option prices in line with FFT/FrFFT results:
    - **K = 2000:** ≈95.25  
    - **K = 2100:** ≈64.83  
    - **K = 2200:** ≈42.95

  - **Wider Intervals:**  
    When wider intervals such as **[-8, 8]** or **[-12, 12]** are used, the results become erratic—prices can be extremely high or even negative. This occurs because the exponential term $e^x$ in the payoff causes the COS coefficients to be mis-scaled in the tails.

- **Practical Consideration:**  
  Although the COS method is theoretically valid, its performance heavily depends on a careful choice of the truncation interval. Selecting an interval that accurately captures the bulk of the probability density of $ x = \ln(S_T/K) $ is crucial for obtaining reliable prices.

---

### Overall Conclusion
- **FFT and FrFFT Methods:**  
  Both methods are robust and provide consistent option prices across different parameter choices and grid sizes. Their implementations adhere well to the theoretical foundations.

- **COS Method:**  
    While it can yield accurate results, the COS method is highly sensitive to the choice of the truncation interval [a, b]. Extra care and calibration are necessary to avoid numerical errors. Its flexibility in handling both call and put options makes it a powerful tool, provided the interval is chosen carefully.


**Summary:** All three transform-based option pricing techniques (FFT, FrFFT, and COS) produce similar prices when implemented correctly and when parameters are chosen appropriately. FFT and FrFFT are robust and stable, whereas the COS method requires extra attention to the truncation interval to ensure reliable outcomes.